# Baseline Model 1: 1D-CNN (Spilka et al., 2016)

**Objective:** Reproduce the Deep Learning baseline using a standard 1D-CNN architecture on FHR signals ONLY (Unimodal).

**Reference:** Spilka, J., et al. (2016). *Deep Learning for Fetal Heart Rate Analysis.*

## 1. Setup & Imports

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import matplotlib.pyplot as plt

# --- GitHub & Colab Setup ---
try:
    from google.colab import userdata
    
    # 1. Clone Repo using Secret Token
    token = userdata.get('GITHUB_AUTH_TOKEN')
    repo_name = "NeuroFetal-AI"
    username = "Krishna200608"
    repo_url = f"https://{token}@github.com/{username}/{repo_name}.git"
    
    if not os.path.exists(repo_name):
        print(f"Cloning {repo_name}...")
        get_ipython().system(f"git clone {repo_url}")
    
    # 2. Configure Git
    os.chdir(repo_name)
    get_ipython().system('git config --global user.email "krishnasikheriya001@gmail.com"')
    get_ipython().system('git config --global user.name "Krishna200608"')
    
    # 3. Install Dependencies
    get_ipython().system('pip install wfdb')

    BASE_DIR = os.getcwd()
    sys.path.append(os.path.join(BASE_DIR, "Code", "scripts"))
    print("Running in Colab (GitHub Integration Active)")

except ImportError:
    # Local Fallback
    BASE_DIR = os.path.abspath(os.path.join("..", ".."))
    sys.path.append(os.path.abspath(os.path.join("..", "scripts")))
    print("Running Locally")

import data_ingestion
print(f"TensorFlow Version: {tf.__version__}")

## 2. Load and Process Data
Using the existing `data_ingestion` pipeline. If data is missing locally, we attempt to run the ingestion script.

In [ ]:
# Parameters
WINDOW_SIZE = 2400  # 20 minutes at 2Hz
STRIDE = 300

PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "Datasets", "processed")
X_path = os.path.join(PROCESSED_DATA_DIR, "X_fhr.npy")
y_path = os.path.join(PROCESSED_DATA_DIR, "y.npy")

# Check if data exists, if not, run ingestion
if not os.path.exists(X_path) or not os.path.exists(y_path):
    print("Processed data not found. Running data_ingestion.py...")
    # This assumes data_ingestion.py handles download/processing
    get_ipython().system(f"python Code/scripts/data_ingestion.py")

try:
    X_signal = np.load(X_path)
    y = np.load(y_path)
    print(f"Loaded Data: X_signal {X_signal.shape}, y {y.shape}")
except FileNotFoundError:
    print("Error: Data ingestion failed or data not found.")
    # Stop execution if critical
    raise

## 3. Define Baseline CNN (Spilka et al.)

In [ ]:
def build_baseline_cnn(input_shape=(2400, 1)):
    inputs = keras.Input(shape=input_shape)
    
    # Block 1
    x = layers.Conv1D(filters=16, kernel_size=7, strides=1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    # Block 2
    x = layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    # Block 3
    x = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling1D()(x)
    
    # Dense Classification Head
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="Baseline_CNN")
    return model

model = build_baseline_cnn(input_shape=(X_signal.shape[1], 1))
model.summary()

## 4. Training (5-Fold CV)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

aucs = []
accs = []

X = X_signal
if X.ndim == 2:
    X = np.expand_dims(X, axis=-1)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print(f"\nStarting Fold {fold+1}...")
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Class weights
    y_train = y_train.astype(int)
    neg, pos = np.bincount(y_train)
    class_weight = {0: 1.0, 1: (neg / pos) if pos > 0 else 1.0}
    
    model = build_baseline_cnn(input_shape=(X.shape[1], 1))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])
    
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_auc', patience=10, mode='max', restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        class_weight=class_weight,
        callbacks=[early_stopping],
        verbose=1
    )
    
    y_pred = model.predict(X_val, verbose=0)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
    
    aucs.append(auc)
    accs.append(acc)
    print(f"Fold {fold+1} Result -> AUC: {auc:.4f}, Acc: {acc:.4f}")
    
    # Save Best Model
    if len(aucs) == 1 or auc > max(aucs[:-1]):
        model_dir = os.path.join(BASE_DIR, "Code", "Baseline", "Models")
        os.makedirs(model_dir, exist_ok=True)
        model_save_path = os.path.join(model_dir, "baseline_paper3_best_cnn.keras")
        model.save(model_save_path)
        print(f"  Saved best model to {model_save_path}")

print("\n=== Final Results ===")
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)
mean_acc = np.mean(accs)
print(f"Mean AUC: {mean_auc:.4f} +/- {std_auc:.4f}")
print(f"Mean Acc: {mean_acc:.4f}")

In [ ]:
# 5. Save Results & Push to GitHub

results_path = os.path.join(BASE_DIR, "Code", "Baseline", "baseline_paper3_results.txt")
with open(results_path, "w") as f:
    f.write(f"Ref: Paper 3 (Spilka 2016) - CNN Baseline (Colab Run)\n")
    f.write(f"Mean AUC: {mean_auc:.4f}\n")
    f.write(f"Std Dev: {std_auc:.4f}\n")
    f.write(f"Mean Acc: {mean_acc:.4f}\n")
print(f"Results saved to {results_path}")

# Git Commit & Push
try:
    if 'google.colab' in sys.modules:
        print("Pushing results to GitHub...")
        # Ensure we are in repo root
        os.chdir(BASE_DIR)
        
        # Configure again just in case
        get_ipython().system('git config --global user.email "krishnasikheriya001@gmail.com"')
        get_ipython().system('git config --global user.name "Krishna200608"')
        
        # Pull latest changes to avoid conflicts
        get_ipython().system('git pull origin main')
        
        # Add relevant files
        get_ipython().system('git add Code/Baseline/Models/*.keras')
        get_ipython().system('git add Code/Baseline/*.txt')
        
        # Commit
        get_ipython().system('git commit -m "Update CNN Baseline Results (Colab)"')
        
        # Push
        get_ipython().system('git push origin main')
        print("Successfully pushed to GitHub!")
except Exception as e:
    print(f"Git Push Failed: {e}")